# C1 문제풀이

auto-mpg.csv 파일을 이용하여 다음의 문제를 푸시오.

문제 1 , 차무게를 기준으로 무거운무게 중간무게 가벼운무게 함수를 만들고 필터링 하세요
(변수명 : 가벼운무게 1_light, 중간무게 2_medium, 무거운무게 3_heavy)
(주석, 차무게를 기준으로 30분위수까지가 가벼운무게, 70분위수가 중간무게 그 이후는 무거운무게로 정의합니다)
문제 2 , 문제1의 답을 이용하여 차무게를 기준으로 배기량을 비교하는 적절한 시각화 그래프를 그리시오. 

문제 3 , 적절한 시각화 그래프를 기준으로 무게에 따른 배기량을 간단하게 분석하시오.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 0) 데이터 불러오기
df = pd.read_csv("auto-mpg.csv")

# (선택) 혹시 모를 타입 문제 대비: 숫자 컬럼을 숫자로 강제 변환
df["weight"] = pd.to_numeric(df["weight"], errors="coerce")
df["displacement"] = pd.to_numeric(df["displacement"], errors="coerce")

# 결측치 제거(분위수/그래프 안정화)
df = df.dropna(subset=["weight", "displacement"]).copy()

# 1) 차무게 기반 구간(가벼움/중간/무거움) 함수 만들기
def weight_group(df):
    """
    차무게(weight)를 기준으로
    30분위수까지: 1_light
    30~70분위수: 2_medium
    70분위수 초과: 3_heavy
    """
    q30 = df["weight"].quantile(0.3)
    q70 = df["weight"].quantile(0.7)

    # 30분위수까지가 가벼운무게
    light = df[df["weight"] <= q30].copy()

    # 70분위수까지가 중간무게 (즉, 30~70)
    medium = df[(df["weight"] > q30) & (df["weight"] <= q70)].copy()

    # 그 이후는 무거운무게
    heavy = df[df["weight"] > q70].copy()

    return light, medium, heavy, q30, q70

# 함수 실행
_1_light, _2_medium, _3_heavy, q30, q70 = weight_group(df)

print("q30 =", q30, "q70 =", q70)
print("1_light:", _1_light.shape, "2_medium:", _2_medium.shape, "3_heavy:", _3_heavy.shape)

# 2) 시각화: 차무게 그룹별 배기량(displacement) 비교 (Boxplot)
data = [_1_light["displacement"], _2_medium["displacement"], _3_heavy["displacement"]]
labels = ["1_light", "2_medium", "3_heavy"]

plt.figure()
plt.boxplot(data, labels=labels)
plt.title("Displacement by Weight Group")
plt.xlabel("Weight Group (by 30th/70th percentiles)")
plt.ylabel("Displacement")
plt.show()

# (선택) 평균도 같이 보고 싶으면:
print("Mean displacement")
print("1_light:", _1_light["displacement"].mean())
print("2_medium:", _2_medium["displacement"].mean())
print("3_heavy:", _3_heavy["displacement"].mean())


# C3 문제풀이

> 원본 데이터

In [ ]:
import pandas as pd
import numpy as np

patients = pd.DataFrame({
    "patient_id": [1, 2, 3, 4, 5, 6],
    "sex": ["F", "M", "F", "M", "F", "M"],
    "age": [34, 58, np.nan, 45, 72, 29],
    "city": ["Seoul", "Busan", "Seoul", "Incheon", "Busan", "Seoul"]
})

visits = pd.DataFrame({
    "visit_id": [1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008],
    "patient_id": [1, 1, 2, 3, 4, 5, 5, 7],
    "dept": ["IM", "ENT", "IM", "DERM", "IM", "ENT", "IM", "IM"],
    "visit_date": pd.to_datetime(["2026-01-02","2026-01-10","2026-01-03","2026-01-04",
                                  "2026-01-11","2026-01-05","2026-01-12","2026-01-08"]),
    "cost": [12000, 18000, 25000, 15000, 22000, 16000, 30000, 21000]
})

---

### 1. 환자 데이터 체크

- 각 행별 결측치 개수를 row_na_cnt 칼럼으로 추가하고
- 각 열별 결측치 개수를 col_na_cnt 로 계산하기

In [ ]:
# Hint. isna().sum(axis=...)

patients['row_na_cnt'] = patients.isna().sum(axis=1)
#patients라는 데이터 프레임에 'row_na_cnt'라는 칼럼을 추가하고 그 칼럼의 값은
#isna() - 데이터가 결측치면 True, 아니면 False로 출력한다.
#.sum - 값이 True로 나오면 1로 계산하라
#axis=1 - 행을 작업하라는 뜻 

print(patients)

   patient_id sex   age     city  row_na_cnt
0           1   F  34.0    Seoul           0
1           2   M  58.0    Busan           0
2           3   F   NaN    Seoul           1
3           4   M  45.0  Incheon           0
4           5   F  72.0    Busan           0
5           6   M  29.0    Seoul           0


In [ ]:
col_na_cnt = patients.drop(columns='row_na_cnt').isna().sum(axis=0)

print(col_na_cnt)

patient_id    0
sex           0
age           1
city          0
dtype: int64


---

### 2. 환자별 진료비 요약 및 상위 환자 찾기

- 환자 데이터를 기준으로 환자 데이터와 방문자 데이터를 한 눈에 볼 수 있게 병합하기
- 환자별 총 진료비(sum)와 방문 횟수(count) 구하기

In [ ]:
df_merged = pd.merge(patients, visits, how='left', on='patient_id')
#pd.merge = 데이터프레임을 병합하는 함수
#두 데이터프레임에 공통으로 들어가있는 patient_id를 기준으로
#how='left': 왼쪽 테이블(patients)의 모든 행을 유지하며 병합한다.

#left merge를 사용하는 이유는 환자데이터에는 있지만 방문기록없는 id6는 포함하고
#방문기록은 있지만 환자데이터에는 없는 id7을 제외하기 위해


print(df_merged)

   patient_id sex   age     city  row_na_cnt  visit_id  dept visit_date  \
0           1   F  34.0    Seoul           0    1001.0    IM 2026-01-02   
1           1   F  34.0    Seoul           0    1002.0   ENT 2026-01-10   
2           2   M  58.0    Busan           0    1003.0    IM 2026-01-03   
3           3   F   NaN    Seoul           1    1004.0  DERM 2026-01-04   
4           4   M  45.0  Incheon           0    1005.0    IM 2026-01-11   
5           5   F  72.0    Busan           0    1006.0   ENT 2026-01-05   
6           5   F  72.0    Busan           0    1007.0    IM 2026-01-12   
7           6   M  29.0    Seoul           0       NaN   NaN        NaT   

      cost  
0  12000.0  
1  18000.0  
2  25000.0  
3  15000.0  
4  22000.0  
5  16000.0  
6  30000.0  
7      NaN  


In [ ]:
patient_summary = df_merged.groupby('patient_id')['cost'].agg(['sum', 'count'])
# patient_id로 그룹을 묶어서, cost 열에 대해 sum(합계)과 count(개수)를 계산한다.


print(patient_summary)

                sum  count
patient_id                
1           30000.0      2
2           25000.0      1
3           15000.0      1
4           22000.0      1
5           46000.0      2
6               0.0      0


auto-mpg.csv 파일을 이용하여 다음의 문제를 푸시오.

문제 1 , 차무게를 기준으로 무거운무게 중간무게 가벼운무게 함수를 만들고 필터링 하세요
(변수명 : 가벼운무게 1_light, 중간무게 2_medium, 무거운무게 3_heavy)
(주석, 차무게를 기준으로 30분위수까지가 가벼운무게, 70분위수가 중간무게 그 이후는 무거운무게로 정의합니다)
문제 2 , 문제1의 답을 이용하여 차무게를 기준으로 배기량을 비교하는 적절한 시각화 그래프를 그리시오. 

문제 3 , 적절한 시각화 그래프를 기준으로 무게에 따른 배기량을 간단하게 분석하시오.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 0) 데이터 불러오기
df = pd.read_csv("auto-mpg.csv")

# (선택) 혹시 모를 타입 문제 대비: 숫자 컬럼을 숫자로 강제 변환
df["weight"] = pd.to_numeric(df["weight"], errors="coerce")
df["displacement"] = pd.to_numeric(df["displacement"], errors="coerce")

# 결측치 제거(분위수/그래프 안정화)
df = df.dropna(subset=["weight", "displacement"]).copy()

# 1) 차무게 기반 구간(가벼움/중간/무거움) 함수 만들기
def weight_group(df):
    """
    차무게(weight)를 기준으로
    30분위수까지: 1_light
    30~70분위수: 2_medium
    70분위수 초과: 3_heavy
    """
    q30 = df["weight"].quantile(0.3)
    q70 = df["weight"].quantile(0.7)

    # 30분위수까지가 가벼운무게
    light = df[df["weight"] <= q30].copy()

    # 70분위수까지가 중간무게 (즉, 30~70)
    medium = df[(df["weight"] > q30) & (df["weight"] <= q70)].copy()

    # 그 이후는 무거운무게
    heavy = df[df["weight"] > q70].copy()

    return light, medium, heavy, q30, q70

# 함수 실행
_1_light, _2_medium, _3_heavy, q30, q70 = weight_group(df)

print("q30 =", q30, "q70 =", q70)
print("1_light:", _1_light.shape, "2_medium:", _2_medium.shape, "3_heavy:", _3_heavy.shape)

# 2) 시각화: 차무게 그룹별 배기량(displacement) 비교 (Boxplot)
data = [_1_light["displacement"], _2_medium["displacement"], _3_heavy["displacement"]]
labels = ["1_light", "2_medium", "3_heavy"]

plt.figure()
plt.boxplot(data, labels=labels)
plt.title("Displacement by Weight Group")
plt.xlabel("Weight Group (by 30th/70th percentiles)")
plt.ylabel("Displacement")
plt.show()

# (선택) 평균도 같이 보고 싶으면:
print("Mean displacement")
print("1_light:", _1_light["displacement"].mean())
print("2_medium:", _2_medium["displacement"].mean())
print("3_heavy:", _3_heavy["displacement"].mean())


# C4 문제풀이

In [ ]:
import pandas as pd
# 주식회사 아이펠 소속 직원에 대한 정보입니다.
data = {
    'Name': ['Kim', 'Lee', 'Park', 'Choi', 'Jung'],
    'Dept': ['IT', 'Sales', 'IT', 'HR', 'Sales'],
    'Score': [85, 92, 78, 88, 95],
    'position':['junior', 'senior', 'manager','senior' , 'manager'],
    'Salary': [5000, 6000, 4500, 4800, 7000],
    'Age': [25, 34, 31, 28, 45]
}

df = pd.DataFrame(data)
df

,Name,Dept,Score,position,Salary,Age
0,Kim,IT,85,junior,5000,25
1,Lee,Sales,92,senior,6000,34
2,Park,IT,78,manager,4500,31
3,Choi,HR,88,senior,4800,28
4,Jung,Sales,95,manager,7000,45


## 질문 1: "전체 직원 중 IT 부서에서 근무하는 사람들만 따로 모아서 보세요.
학습 도구: .loc[행, 열] 사용법

In [ ]:
it_people = df.loc[df["Dept"] == "IT", :]   # 행: 조건 / 열: 전체(:)
print(it_people)

   Name Dept  Score position  Salary  Age
0   Kim   IT     85   junior    5000   25
2  Park   IT     78  manager    4500   31


## 질문 2: "성적이 90점 이상인 우수 사원들의 이름과 연봉 정보만 콕 집어서 리스트로 만드세요. 
1) loc 활용
2) Dept 칼럼의 값이 'IT'인 데이터를 찾는다.
3) 코드표현 df['Dept'] =='IT'
4) 조립: df.loc[행 조건, 열 선택]

In [ ]:
excellent_list = df.loc[df["Score"] >= 90, ["Name", "Salary"]].values.tolist()
print(excellent_list)

[['Lee', 6000], ['Jung', 7000]]


## 질문 3: "기준 연봉을 5,000으로 설정했을 때, 이보다 많이 받는 사람들을 변수를 활용해 필터링하세요.
학습 도구: .query()

In [ ]:
base_salary = 5000
high_salary = df.query("Salary > @base_salary")
print(high_salary)

   Name   Dept  Score position  Salary  Age
1   Lee  Sales     92   senior    6000   34
4  Jung  Sales     95  manager    7000   45


## 질문 4 : 각 부서의 평균 나이와 평균 점수 그리고 판매금액의 최댓값 최소값 평균값을 데이터 프레임으로 만들어보세요.

In [ ]:
dept_summary = df.groupby("Dept").agg(
    avg_age=("Age", "mean"),
    avg_score=("Score", "mean"),
    salary_max=("Salary", "max"),
    salary_min=("Salary", "min"),
    salary_mean=("Salary", "mean")
)
print(dept_summary)

       avg_age  avg_score  salary_max  salary_min  salary_mean
Dept                                                          
HR        28.0       88.0        4800        4800       4800.0
IT        28.0       81.5        5000        4500       4750.0
Sales     39.5       93.5        7000        6000       6500.0


## 질문 5: Score가 85 이상인 사람만 남긴 뒤, Dept별 Salary 평균을 구하세요.

In [ ]:
dept_salary_mean = (
    df.loc[df["Score"] >= 85]
      .groupby("Dept")["Salary"]
      .mean()
      .to_frame(name="avg_salary")
)
print(dept_salary_mean)

       avg_salary
Dept             
HR         4800.0
IT         5000.0
Sales      6500.0


## 질문 6: "각 부서에 속한 직원의 직급별 연봉을 피벗테이블로 표시하고, 해당 합계(Total Salary)를 표시하세요.
(결측치는 0으로 처리)

In [ ]:
pivot_salary = pd.pivot_table(
    df,
    index="Dept",
    columns="position",
    values="Salary",
    aggfunc="sum",
    fill_value=0
)

pivot_salary["Total Salary"] = pivot_salary.sum(axis=1)   # 부서별 합계 열 추가
pivot_salary.loc["Total"] = pivot_salary.sum(axis=0)      # 전체 합계 행 추가

print(pivot_salary)

position  junior  manager  senior  Total Salary
Dept                                           
HR             0        0    4800          4800
IT          5000     4500       0          9500
Sales          0     7000    6000         13000
Total       5000    11500   10800         27300


# M1 문제 풀이입니다. 
## 1. Data Frame으로 만들어라
## 2. 특정 Column으로 정렬(sort)하라

In [ ]:
# M1 문제 풀이

import pandas as pd

data = {
   'Name' : ['김경국', '김예원', '안선미', '유지숙'], 
   'Age' : [28, 24, 26, 30],
   'gender' : ['남', '여', '여', '여'],
   'City' : ['경기', '인천', '대구','대전']
    
}

df1 = pd.DataFrame(data)
print("변경 전 \n", df1)
print()

# 변경 후 문제 : Age 변경
df_sorted = df1.sort_values('Age') # .sort 메서드 사용하여 정렬하였습니다.
print("변경 후 \n",df_sorted)



변경 전 
   Name  Age gender City
0  김경국   28      남   경기
1  김예원   24      여   인천
2  안선미   26      여   대구
3  유지숙   30      여   대전

변경 후 
   Name  Age gender City
1  김예원   24      여   인천
2  안선미   26      여   대구
0  김경국   28      남   경기
3  유지숙   30      여   대전


# M2 문제풀이

## 문제 : 변환(Transformation)
각 class 그룹별로 fare의 누적합(cumulative sum) 을 계산하여 fare_cumsum 컬럼을 새로 생성하세요. 이 질문의 코드 인데요. 빈칸에는 무엇이 들어갈까요?
각 class 그룹 내에서 age를 z-score로 표준화하여 age_zscore 컬럼을 생성하세요. 아래 코드에 '=' 뒤에 올 코드는 무엇일까요?

In [ ]:
import pandas as pd
import numpy as np

np.random.seed(0)

# 승객 데이터를 가정한 500개의 행을 가진 예제 데이터 생성
data = {
    'class': np.random.choice(['First', 'Second', 'Third'], 500),
    'fare': np.random.uniform(20, 150, 500),
    'age': np.random.normal(35, 8, 500)
}

df = pd.DataFrame(data)
display(df)

,class,fare,age
0,First,83.806420,42.112162
1,Second,81.034779,36.593128
2,First,134.752208,20.730669
3,Second,83.493308,48.702986
4,Second,73.893773,27.293905
...,...,...,...
495,First,24.195527,23.662590
496,First,79.369387,38.399735
497,Third,127.223702,34.876334
498,Second,106.918760,41.370661


In [ ]:
# 1. 각 class 그룹별로 fare의 누적합(cumulative sum) 을 계산하여
# fare_cumsum 컬럼을 새로 생성하세요. 이 질문의 코드 인데요.
# 빈칸에는 무엇이 들어갈까요?
# 힌트! 새로 생성할 컬럼 이름을 자세히 봐볼까요?

df['fare_cumsum'] = (
    df.groupby('class')['fare']
      .cumsum()
)

display(df)

,class,fare,age,fare_cumsum
0,First,83.806420,42.112162,83.806420
1,Second,81.034779,36.593128,81.034779
2,First,134.752208,20.730669,218.558627
3,Second,83.493308,48.702986,164.528087
4,Second,73.893773,27.293905,238.421860
...,...,...,...,...
495,First,24.195527,23.662590,15183.289331
496,First,79.369387,38.399735,15262.658718
497,Third,127.223702,34.876334,12609.883194
498,Second,106.918760,41.370661,14225.539231


In [ ]:
# 2. 각 class 그룹 내에서 age를 z-score로 표준화하여 age_zscore 컬럼을 생성하세요.
# 아래 코드에 '=' 뒤에 올 코드는 무엇일까요?
# 힌트! 1. z-score 공식 : (값 - 평균) / 표준편차
# 힌트! 2. transform()함수와 lambda함수 활용

df['age_zscore'] = df.groupby('class')['age'].transform(
    lambda x: (x - x.mean()) / x.std()      # “x 안에 있는 값들을 평균을 빼고,표준편차로 나눈 값으로 바꿔라”
)


display(df)

,class,fare,age,fare_cumsum,age_zscore
0,First,83.806420,42.112162,83.806420,0.989768
1,Second,81.034779,36.593128,81.034779,0.160855
2,First,134.752208,20.730669,218.558627,-1.834599
3,Second,83.493308,48.702986,164.528087,1.609268
4,Second,73.893773,27.293905,238.421860,-0.951390
...,...,...,...,...,...
495,First,24.195527,23.662590,15183.289331,-1.447310
496,First,79.369387,38.399735,15262.658718,0.499378
497,Third,127.223702,34.876334,12609.883194,-0.110691
498,Second,106.918760,41.370661,14225.539231,0.732277


# M3 문제풀이

## 부서별 평균 점수와 인원 수 구하기

In [ ]:
import pandas as pd

students = pd.DataFrame({
    "dept": ["AI", "AI", "AI", "Data", "Data", "Design", "Design", "Design"],
    "name": ["민수", "지민", "서연", "철수", "영희", "유나", "지수", "현우"],
    "score": [88, 95, 76, 90, 84, 70, 92, 85],
    "hours": [12, 15, 9, 14, 11, 8, 16, 10]
})

print(students)

     dept name  score  hours
0      AI   민수     88     12
1      AI   지민     95     15
2      AI   서연     76      9
3    Data   철수     90     14
4    Data   영희     84     11
5  Design   유나     70      8
6  Design   지수     92     16
7  Design   현우     85     10


### ✅ dept 기준으로 그룹화해서 아래를 구하세요.
#### 부서별(dept) score 평균, hours 평균, 인원 수(학생 수)

### ✅ 결과를 dept_summary라는 변수에 저장하고 출력하세요.
#### groupby를 반드시 사용할 것, 표 형태로 나오게 할 것

In [ ]:
# 1) dept 기준으로 그룹 만들기(묶기)
g = students.groupby("dept")

# 2) 그룹별로 여러 통계를 한 번에 계산해서 표(DataFrame) 만들기
dept_summary = g.agg(
    score_mean=("score", "mean"),   # score 평균
    hours_mean=("hours", "mean"),   # hours 평균
    headcount=("name", "count")     # 인원 수(이름 개수)
)

# 3) 결과 출력
print(dept_summary)

        score_mean  hours_mean  headcount
dept                                     
AI       86.333333   12.000000          3
Data     87.000000   12.500000          2
Design   82.333333   11.333333          3
